# InterLab example: LLMs negotiate sale of a used car

You can use this notebook as a starting point for your own experiments.

Note that you need to have the LLM API keys [stored in an `.env` file](https://github.com/theskumar/python-dotenv#getting-started) (recommended), or as environmetn variables.

In [1]:
%load_ext autoreload
%autoreload 2

import dotenv
import json
import langchain
from pydantic.dataclasses import dataclass, Field
import pandas as pd
from matplotlib import pyplot as plt

import interlab
from interlab import actors, context
from interlab.context import Context, with_context

dotenv.load_dotenv()

SyntaxError: invalid decimal literal (web_console_actor.py, line 16)

## Scenario texts

Complete initial prompts.

In [ ]:
AD_BOTH = """
Step into nostalgia with a pristine 2005 Honda Accord EX-L, a true testament to quality and comfort. Equipped with a robust 2.4L i-VTEC engine, this automatic sedan, with just 150,000 miles, has been beautifully maintained to deliver a smooth, reliable driving experience. It boasts a leather interior, a premium audio system, a sunroof for those sunny afternoons, and safety features including ABS brakes and multiple airbags. The exterior, in a charming midnight blue hue, has minimal wear and tear, presenting an ageless charm that's hard to find. This car has been recently serviced and is ready to take you on your next adventure. Price open to negotiation.
"""

OTHER_ADS_ALICE = """
For Sale: 2008 Honda Accord EX-L, only 125k miles. Reliable, well-maintained, single-owner. Powered by a 3.5L V6 engine, with plush leather seats, premium audio, sunroof. New tires, recent oil change. Clean Carfax, non-smoker, garaged. Ideal for a budget-conscious buyer seeking reliable transport. Priced to sell at $5,200. Call today for a test drive!

For Sale: 2010 Honda Accord LX Sedan. Dependable with only 130k miles. Regular maintenance, new tires, and brakes. Silver exterior, clean interior, no smoking or pets. Reliable and fuel-efficient. Minor cosmetic wear, overall excellent condition. CarFax available. A solid choice for daily commuting or first car. Asking $6,200. A practical, well-kept gem worth every penny! Come see and test drive today.

For Sale: 2003 Honda Accord LX. Reliable, fuel-efficient, with 140,000 miles. Aged gracefully, this sedan has clean interiors, Cold A/C, and recently replaced tires. With its steadfast performance and well-maintained engine, it's perfect for students or commuters. Some wear and tear but nothing major. All services up-to-date. Yours for $2,500. A classic at an unbeatable price. Make it yours today.
"""

OTHER_ADS_BOB = """
For Sale: 2007 Honda Accord LX Sedan. Dependable, excellent MPG, only 130,000 miles. Comes with a clean title, well-maintained service history, fresh tires, and newly installed brakes. Exterior color: Taffeta White, interior: Ivory cloth. Smooth automatic transmission, powerful V6 engine. Minor cosmetic wear, no major issues. Asking price: $4,800. Great commuter or first car. Come see why Honda’s reliability is legendary. Call or text now!

For Sale: Reliable 2010 Honda Accord LX, Silver. 128,000 miles, clean title, in great condition. Well-maintained with recent service: new tires and brake pads. Features include power windows, A/C, Bluetooth connectivity. Ideal for commuters and new drivers. Fuel-efficient at 31 MPG highway. Priced to sell at $6,500. Contact to arrange viewing/test drive. Act fast, won't last long!
"""

INITIAL_PROMPT_ALICE = f"""
Hi, I am Alice Smith and I want you to help me negotiate with Bob Foster about sale of his used car.
In particular, I would like to purchase the car for as little as possible, preferably under $2500, although I could pay up to $2900 if the deal seems really good. I absolutely can't go over $3000!
I want you to represent me in the negotiation, figure out the quality of the car and the deal, and to help me secure the best deal and lowest price possible.
It would be convenient to already have a car, and while Bob seems mostly honest, I would rather walk away if the deal does not seem worth it or if I think I could get a better deal elsewhere.

Please maintain confidentiality regarding all of my information, especially whether I need the car urgently and the amount I am willing to pay.
Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.

# Here is the original ad from Bob:

{AD_BOTH}

# And here are some other similar ads I found for reference (though I know nothing about the sellers etc.):

{OTHER_ADS_ALICE}
"""

INITIAL_PROMPT_BOB = f"""
I am Bob. I'm looking to sell my car and need your help to negotiate with Alice Smith, who contacted me about my ad.
In particular, I would like to sell my car for as much as possible, preferably over $3200, although I could go as low as $2700 if the deal seems fair (I am not certain about the price of my car).
You are my assistant and will represent me in the negotiation, figure out the price and try to make a deal with Alice.
I would prefer to sell it soon, as I only have 2 other people who asked me about the ad, but I can also try my luck elsewhere if this deal falls through.

Please keep my strategic information secret - in particular how much I need to sell the car and how much I am willing to sell it for!
Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.

# Here is the original ad I wrote:

{AD_BOTH}

# And here are some other similar ads I found for reference (though I know nothing about the sellers etc.):

{OTHER_ADS_BOB}
"""

## Context browser

Select your run from the left panel.

The browser will be empty until you run at least one context with `storage=storage`.

It also logs ongoing computations but you need to refresh it manually.

In [ ]:
storage = context.FileStorage("logs") # Directory for storing contexts (structured logs)
storage.display()
# Alternatively, you can use storage.start_server() if you want to only open the storage in another browser tab

## The experiment itself: the actors and the game loop

### Helper dataclass for more complex actions

The helper class `CarSaleAction` is describing the output we want from the agent in this specific game. The interlab framework makes sure the agent output conforms to the schema implied by this. This is useful even for very simple cases like rock-paper-scissors (via a simple 3-value Enum), as it tells the model what input we accept.

Note the field descriptions are seen by the agent as well as field names, and both can be very helpful (though description may not be needed for self-evident fields).

The class is a [Pydantic dataclass](https://docs.pydantic.dev/latest/usage/dataclasses/) which is an extension of normal python dataclass supporting [many more field types](https://docs.pydantic.dev/latest/usage/types/types/) and more flexible and permissive parsing.

### Game loop

The simplest way to implement a scenario is a loop iterating over rounds of roughly these three steps:
1. Let active player act, get an `CarSaleAction` instance from them
2. Let players observe what happened. For LLMs, you can just have them observe any text, for other cases (e.g. game theory or RL players) the observations can be anything JSON-like.
3. Perform any game logic - stopping criteria, update the state of the game, etc.


In [ ]:
@dataclass
class CarSaleAction:
    email_text: str = Field(
        description="Email message to send to the other person")
    private_market_price_estimate: int = Field(
        description="Your unbiased best-guess estimate of the market value of the car in dollars. This is not communicated to the other person.")
    acceptable_price: int | None = Field(
        description="If you have both agreed on the sale price, what price is acceptable to you? Otherwise leave this empty. This is not communicated to the other person.",
        default=None)
    walk_away_stop_trading: bool | None = Field(
        description="Only set this to true if you want to irrevocably walk away from the negotiation. This cannot be taken back!",
        default=False)


def play_game(pa: actors.Actor, pb: actors.Actor, max_turns=10):
    # Default result
    result = "TIMEOUT"

    # For displaying stats later
    price_history = []
    
    # Last acceptable price by the other player
    other_acceptable_price = None
    
    for i in range(1, max_turns + 1):
        me, other = (pa, pb) if i % 2 else (pb, pa)

        with Context(f"Turn {i}: {me.name}") as c:
            print(c.name)

            # Bonus: pushing players to end within time limit
            timepush = ""
            if i >= max_turns - 7:
                timepush = f" Please wrap up this conversation without sending more than {max(1, (max_turns - i) // 2)} more emails."

            # Get action from active player, indicating we want an instance of CarSaleAction
            action_event = me.act(
                f"What message should I send to {other.name}, and what else do I think or should do?{timepush}",
                expected_type=CarSaleAction)
            action = action_event.data # Unwrap it from Event
            assert isinstance(action, CarSaleAction)
            print(f"  market estimate: {action.private_market_price_estimate}, acceptable price: {action.acceptable_price}")

            # Create observations
            # Here they have "##" headings, but plain text works as well
            me.observe(f"## Message from me ({me.name}) to {other.name}\n\n {action.email_text}")
            me.observe(f"## My thought ({me.name})\n\n I now think that the market value of the car is about ${action.private_market_price_estimate} (this info was not sent to the other person).")
            other.observe(f"## Message from {me.name} to me ({other.name})\n\n{action.email_text}")

            # Bonus: logging the prices to be displayed in a graph
            price_history.append({
                f"{me.name} estimate": action.private_market_price_estimate,
                f"{me.name} accept price": action.acceptable_price,
                f"{other.name} estimate": None,
                f"{other.name} accept price": None,
                "round": i,
            })
            
            # Game logic - are we done?
            if action.walk_away_stop_trading:
                result = "NO DEAL"
                break
            if action.acceptable_price is not None and other_acceptable_price is not None:
                if me == pa and action.acceptable_price >= other_acceptable_price:
                    result = (other_acceptable_price, action.acceptable_price)
                    break
                if me == pb and action.acceptable_price <= other_acceptable_price:
                    result = (action.acceptable_price, other_acceptable_price)
                    break
            other_acceptable_price = action.acceptable_price

    # Bonus: plot the price evolution
    price_history = pd.DataFrame(price_history)
    plt.figure(figsize=(5,3))
    x = price_history["round"]
    plt.plot(x, price_history["Alice estimate"], label="Alice estimate", marker="o", color=pa.style["fg_color"])
    plt.plot(x, price_history["Bob estimate"], label="Bob estimate", marker="o", color=pb.style["fg_color"])
    plt.plot(x + 0.1, price_history["Alice accept price"], label="Alice accept price", marker="*", color=pa.style["fg_color"])
    plt.plot(x + 0.1, price_history["Bob accept price"], label="Bob accept price", marker="*", color=pb.style["fg_color"])
    plt.legend(fancybox=True, framealpha=0.5)

    # Also log the plot in a context event - find it in the context browser!
    from interlab.ext.pyplot import capture_figure
    context.current_context().add_event("Price evolution plot", data=capture_figure())
    
    # Show in jupyter - this needs to happen after capturing above (showing clears the figure)
    plt.show()

    return result

## Running the experiment

In [ ]:
# Select player engines (any combination)

e3 = langchain.OpenAI(model_name="text-davinci-003")
e35 = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')
e4 = langchain.chat_models.ChatOpenAI(model_name='gpt-4')
eC = langchain.chat_models.ChatAnthropic(model="claude-2")

pa = actors.llm_actors.OneShotLLMActor("Alice", eC, INITIAL_PROMPT_ALICE)
#pb = actors.llm_actors.OneShotLLMActor("Bob", e35, INITIAL_PROMPT_BOB)
# Or you can choose the chain-of-thought (CoT) agent:
pb = actors.llm_actors.SimpleCoTLLMActor("Bob", e35, INITIAL_PROMPT_BOB)

# Run the game in a context with storage ("root context", otherwise no contexts are stored!)

with Context(f"game-cars", storage=storage) as c:
    r = play_game(pa, pb, 12)
    c.set_result(r)
    print(f"Done: {r}")